### One-way ANOVA (일원분산분석)

#### #01. 작업준비

##### 패키지 가져오기

In [3]:
from pandas import read_excel, melt
from scipy.stats import shapiro, normaltest, ks_2samp, bartlett, fligner, levene, f_oneway, chi2_contingency
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
import sys
import os

sys.path.append(os.path.dirname(os.path.dirname(os.getcwd())))
from helper import equal_variance_test

ImportError: cannot import name 'equal_variance_test' from 'helper' (d:\jihun5\jupyter\helper.py)

In [ ]:
# 데이터 가져오기
# 어떤 지역에서 동일한 제품을 판매하고 있는 두 가게에 대한 일별 매출액
df = read_excel("https://data.hossam.kr/E02/store.xlsx")
df


NameError: name 'read_excel' is not defined